In [3]:
!conda init

no change     /home/ces/anaconda3/condabin/conda
no change     /home/ces/anaconda3/bin/conda
no change     /home/ces/anaconda3/bin/conda-env
no change     /home/ces/anaconda3/bin/activate
no change     /home/ces/anaconda3/bin/deactivate
no change     /home/ces/anaconda3/etc/profile.d/conda.sh
no change     /home/ces/anaconda3/etc/fish/conf.d/conda.fish
no change     /home/ces/anaconda3/shell/condabin/Conda.psm1
no change     /home/ces/anaconda3/shell/condabin/conda-hook.ps1
no change     /home/ces/anaconda3/lib/python3.11/site-packages/xontrib/conda.xsh
no change     /home/ces/anaconda3/etc/profile.d/conda.csh
no change     /home/ces/.bashrc
No action taken.


In [8]:
!conda env list 



# conda environments:
#
base                  *  /home/ces/anaconda3
Spark                    /home/ces/anaconda3/envs/Spark
argame                   /home/ces/anaconda3/envs/argame
image_processing         /home/ces/anaconda3/envs/image_processing
mlflow                   /home/ces/anaconda3/envs/mlflow
object_detection         /home/ces/anaconda3/envs/object_detection
plotly                   /home/ces/anaconda3/envs/plotly
spark_                   /home/ces/anaconda3/envs/spark_



In [6]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Sample data (replace with your data or load from a CSV/database)
data = {
    "City": ["Mexico City", "Guadalajara", "Monterrey", "Cancun", "Tijuana"],
    "Latitude": [19.4326, 20.6597, 25.6866, 21.1619, 32.5149],
    "Longitude": [-99.1332, -103.3496, -100.3161, -86.8515, -117.0382],
    "Population": [8918653, 1495182, 1135512, 888797, 1300983],
    "Category": ["Metropolitan", "Metropolitan", "Industrial", "Tourism", "Border"],
}

# Create a DataFrame
df = pd.DataFrame(data)

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Mexico Dashboard", style={"textAlign": "center"}),
    
    # Filters
    html.Div([
        html.Label("Filter by Category:"),
        dcc.Dropdown(
            id="category-filter",
            options=[{"label": cat, "value": cat} for cat in df["Category"].unique()],
            value=None,
            multi=True,
            placeholder="Select a category"
        ),
        html.Label("Filter by Population Range:"),
        dcc.RangeSlider(
            id="population-slider",
            min=df["Population"].min(),
            max=df["Population"].max(),
            step=100000,
            marks={i: str(i) for i in range(df["Population"].min(), df["Population"].max() + 1, 500000)},
            value=[df["Population"].min(), df["Population"].max()]
        ),
    ], style={"margin": "20px"}),
    
    # Map
    dcc.Graph(id="mexico-map"),
])

# Callback to update the map based on filters
@app.callback(
    Output("mexico-map", "figure"),
    [Input("category-filter", "value"),
     Input("population-slider", "value")]
)
def update_map(selected_categories, population_range):
    # Filter data based on selected categories
    filtered_df = df.copy()
    if selected_categories:
        filtered_df = filtered_df[filtered_df["Category"].isin(selected_categories)]
    
    # Filter data based on population range
    filtered_df = filtered_df[
        (filtered_df["Population"] >= population_range[0]) &
        (filtered_df["Population"] <= population_range[1])
    ]
    
    # Create the map
    fig = px.scatter_mapbox(
        filtered_df,
        lat="Latitude",
        lon="Longitude",
        hover_name="City",
        hover_data=["Population", "Category"],
        zoom=4,
        height=600,
        title="Map of Mexico"
    )
    
    # Customize the map style
    fig.update_layout(
        mapbox_style="open-street-map",
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
    )
    
    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)

ModuleNotFoundError: No module named 'dash'